This is a practice model to get the steps involved in machine learning.


we are going to 
<ol>
<li>Look at the big picture</li>
<li>Get data</li>
<li>Dscover the data to gain insights</li>
<li>Prepare the data for the machine learning model</li>
<li>Select a model and train it</li>
<li>Finetune the model</li>
<li>Present the solution</li>
<li>Launch the system</li>
</ol>

# Looking at the big picture
## Describing the problem


In this example, we use the California Housin Prices datasets. it is based on the 1990 California cencus.

The first task is to build a model of housing prices in Cali.


District prices are estimated manually which is time consuming. The boss wants a model that can predict the median housing price in a district given data about the district. 


The problem requires the building of a 
<ul>
<li>Supervised -> The data is labelled with the true median value</li>
<li>Regression -> We are asked to predict a value</li>
<li>Multiple regression-> We will use multiple features to make a prediction</li>
<li>Univariate regression -> We are using going to predict a single value for each district</li>
</ul>


## Selecting a performance metric

A typical performance measure for regression problems is the Root Mean Square Error

RMSE is a cost function. 



# Get the data

we have the dataset in the data folder. Next, we have to load the dataset into a pandas dataframe

In [1]:
import pandas as pd

In [2]:
housing = pd.read_csv("data/housing.csv")

In [3]:
#check the first instances of the housing data dataframe
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


Each row represents one block. There are 10 attributes. One can use the info methos to get a quick description of the data.

In [4]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


### Things to note about the dataset

<ul>
<li> <strong>NB</strong>: You notice that the total bedrooms feature has 20433 as opposed to 20640. Will be taken care of.</li>
<li> All features have numerical values except the ocean proximity feature. The following code shows the feature </li>

In [5]:
#value counts function helps us know how many instances belong to each category.

housing["ocean_proximity"].value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

stopped at page 51